In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetV2B0 
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from joblib import dump 

In [ ]:
INPUT_TSV = '\dataset\znacajke.tsv'
TRAIN_TSV = '\dataset\train_21.tsv'
TEST_TSV = '\dataset\test_21.tsv'
SPECTROGRAM_PATH = '\dataset\mel_spektrogrami'

IMG_HEIGHT = 160 
IMG_WIDTH = 160
BATCH_SIZE = 32
INITIAL_EPOCHS = 20 
FINE_TUNE_EPOCHS = 10 

MODEL_SAVING_PATH = 'final_age_cnn_model.keras'

In [ ]:
datafile_all = pd.read_csv(INPUT_TSV, sep='\t')

def path_correction(old_path):
    file_n = os.path.basename(old_path)
    return os.path.join(SPECTROGRAM_PATH, file_n)
datafile_all['spectrogram_path'] = datafile_all['spectrogram_path'].apply(path_correction)

older = ['sixties', 'seventies', 'eighties', 'nineties']
datafile_all['age'] = datafile_all['age'].replace(older, '60plus')
younger = ['teens', 'twenties']
datafile_all['age'] = datafile_all['age'].replace(younger, 'twentiesAndUnder')

le = LabelEncoder()
datafile_all['age_encoded'] = le.fit_transform(datafile_all['age'])
dump(le, 'age_label_encoder_twenties_under.joblib')

df_cnn = datafile_all[['spectrogram_path', 'age_encoded', 'person_id']].copy()
df_train_ids = pd.read_csv(TRAIN_TSV, sep='\t')
df_test_ids = pd.read_csv(TEST_TSV, sep='\t')
train_person_ids = df_train_ids['person_id'].unique()
test_person_ids = df_test_ids['person_id'].unique()
train_df = df_cnn[df_cnn['person_id'].isin(train_person_ids)]
test_df = df_cnn[df_cnn['person_id'].isin(test_person_ids)]

train_image_paths = train_df['spectrogram_path'].values
train_labels = train_df['age_encoded'].values
test_image_paths = test_df['spectrogram_path'].values
test_labels = test_df['age_encoded'].values

print(f"\nNumber of new age classes: {len(le.classes_)}")

In [ ]:
def load_and_preprocess_image(path, label):
    image = tf.io.read_file(path)
    image = tf.io.decode_png(image, channels=3)
    image = tf.image.resize(image, [IMG_HEIGHT, IMG_WIDTH])
    return image, label

train_dataset = tf.data.Dataset.from_tensor_slices((train_image_paths, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_image_paths, test_labels))

AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.shuffle(buffer_size=1000).batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)


In [ ]:
IMG_SHAPE = (IMG_HEIGHT, IMG_WIDTH, 3)
num_classes = len(le.classes_) 

data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1)
], name="data_augmentation")

base_model = EfficientNetV2B0(include_top=False, weights='imagenet', input_shape=IMG_SHAPE)
base_model.trainable = False
preprocessing_layer = tf.keras.applications.efficientnet_v2.preprocess_input

inputs = keras.Input(shape=IMG_SHAPE)
x = data_augmentation(inputs)
x = preprocessing_layer(x)
x = base_model(x, training=False) 
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)
model = keras.Model(inputs, outputs)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6)

print("First phase of training: ")
history = model.fit(
    train_dataset,
    epochs=INITIAL_EPOCHS,
    validation_data=test_dataset,
    callbacks=[early_stopping, reduce_lr]
)

In [ ]:
base_model.trainable = True
fine_tune_at = -40 

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

optimizer_fine_tune = tf.keras.optimizers.Adam(learning_rate=1e-5)
model.compile(optimizer=optimizer_fine_tune, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

print("\nSeconf phase, fine-tuning.")

total_epochs = INITIAL_EPOCHS + FINE_TUNE_EPOCHS
history_fine_tune = model.fit(
    train_dataset,
    epochs=total_epochs,
    initial_epoch=history.epoch[-1],
    validation_data=test_dataset,
    callbacks=[early_stopping, reduce_lr]
)

In [ ]:
model.save(MODEL_SAVING_PATH)
print(f"Model saved in: '{MODEL_SAVING_PATH}'")

acc = history.history['accuracy'] + history_fine_tune.history['accuracy']
val_acc = history.history['val_accuracy'] + history_fine_tune.history['val_accuracy']
loss = history.history['loss'] + history_fine_tune.history['loss']
val_loss = history.history['val_loss'] + history_fine_tune.history['val_loss']

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(acc, label='Accuracy in training')
plt.plot(val_acc, label='Accuracy on validation')
plt.axvline(x=len(history.history['accuracy'])-1, color='r', linestyle='--', label='Fine-tune start')
plt.legend(loc='lower right')
plt.title('Accuracy during training')

plt.subplot(1, 2, 2)
plt.plot(loss, label='Loss in training')
plt.plot(val_loss, label='Loss in validation')
plt.axvline(x=len(history.history['loss'])-1, color='r', linestyle='--', label='Fine-tune start')
plt.legend(loc='upper right')
plt.title('Loss during training')
plt.show()

In [ ]:
print("\nFinal evaluation: ")
loss, accuracy = model.evaluate(test_dataset)
print(f"Accuracy on the test set: {accuracy*100:.2f}%")

y_pred_probs = model.predict(test_dataset)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.concatenate([y for x, y in test_dataset], axis=0)

print("\nDetailed class report:")
print(classification_report(y_true, y_pred, target_names=le.classes_, zero_division=0))

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Purples', xticklabels=le.classes_, yticklabels=le.classes_)
plt.title('Confusion matrix')
plt.ylabel('Actual class')
plt.xlabel('Predicted class')
plt.show()